# Helper Functions

## Imports

In [155]:
import boto3
import json
import pytz
import seaborn as sea
import requests 
import mplfinance as mpf
import pandas as pd
from zoneinfo import ZoneInfo
from datetime import datetime, timedelta

## Credentials and dates

In [50]:
def get_secret():
    client = boto3.client('secretsmanager', region_name='us-west-2')
    return json.loads(client.get_secret_value(SecretId="tokens")['SecretString'])

def parse_time(iso_string ):
    utc_time = datetime.fromisoformat(iso_string.replace("Z", "+00:00"))
    portland_time = utc_time.astimezone(ZoneInfo("America/Los_Angeles"))
    return portland_time

secret = get_secret()
alpaca_key = secret['alpaca_key']
alpaca_secret = secret['alpaca_secret']
polygon_key = secret['polygon_key']

## Plotting

In [22]:
def plot_candles(data):
    df = pd.DataFrame(data)
    df['t'] = pd.to_datetime(df['t'])  # Convert timestamp to pandas datetime
    df.set_index('t', inplace=True)  # Set datetime as index
    df = df.rename(columns={'ha_o': 'Open', 'ha_h': 'High', 'ha_l': 'Low', 'ha_c': 'Close'})  # Rename for mplfinance
    
    # Plot using mplfinance
    mpf.plot(df, type='candle', style='charles', title='Heikin-Ashi Chart', ylabel='Price')
    return 

## Heikin Ashi

In [97]:
def calculate_heikin_ashi(data):

    heikin_ashi = []
    previous_ha_close = (data[0]['o'] + data[0]['h'] + data[0]['l'] + data[0]['c']) / 4
    previous_ha_open = (data[0]['o'] + data[0]['c']) / 2

    for candle in data:
        ha_close = (candle['o'] + candle['h'] + candle['l'] + candle['c']) / 4
        ha_open = (previous_ha_open + previous_ha_close) / 2
        ha_high = max(candle['h'], ha_open, ha_close)
        ha_low = min(candle['l'], ha_open, ha_close)
        heikin_ashi.append({
            't': candle['t'],
            'ha_o': ha_open,
            'ha_h': ha_high,
            'ha_l': ha_low,
            'ha_c': ha_close
        })

        previous_ha_open = ha_open
        previous_ha_close = ha_close

    return heikin_ashi

## Getting Data

### Candles

In [115]:
def get_candles(ticker, date):
    base_url = "https://data.alpaca.markets/v2/stocks"
    headers = {
        "APCA-API-KEY-ID": alpaca_key,
        "APCA-API-SECRET-KEY": alpaca_secret
    }
    
    start_time = datetime.strptime(date, "%Y-%m-%d")
    end_time = start_time + timedelta(days=1)
    today = datetime.today()  # Use UTC to match API expectations

    params = {
        "start": start_time.isoformat() + "Z",  # Format as ISO 8601
        "timeframe": "30Min"  # Adjust timeframe as needed (e.g., "1Min", "5Min", "1Day")
    }

    # Add the 'end' parameter only if end_time is before today
    if end_time < today:
        params["end"] = end_time.isoformat() + "Z"  # Format as ISO 8601

    url = f"{base_url}/{ticker}/bars"
    r = requests.get(url, headers=headers, params=params)
    
    if r.status_code == 200:
        data = r.json().get('bars', [])
        for row in data:
            row['t'] = parse_time(row['t'])

        # Define trading hours in UTC
        trading_start = datetime.strptime("09:30", "%H:%M").replace(tzinfo=pytz.timezone("US/Eastern"))
        trading_end = datetime.strptime("16:00", "%H:%M").replace(tzinfo=pytz.timezone("US/Eastern"))

        # Filter for only regular trading hours
        filtered_data = []
        for row in data:
            candle_time = row['t'].astimezone(pytz.timezone("US/Eastern"))
            if trading_start.time() <= candle_time.time() <= trading_end.time():
                filtered_data.append(row)

        return filtered_data
    else:
        r.raise_for_status()


### Dividend Dates

In [ ]:
def get_dividend_dates(ticker):
    url = f"https://api.polygon.io/v3/reference/dividends?ticker={ticker}&apiKey={polygon_key}"
    results = []
    
    while url:
        if 'apiKey' not in url:
            url = url + f'&apiKey={polygon_key}'
        r = requests.get(url)
        
        if r.status_code != 200:
            raise Exception(f"Failed to fetch data: {r.status_code} - {r.text}")
        
        data = r.json()
        results.extend(data.get('results', []))
        
        url = data.get('next_url')
    
    return results

### One Day Profitability (Uses Candles)

In [121]:
def get_day_delta(ticker, date):
    data  = get_candles(ticker, date)
    return data[-1]['vw'] - data[0]['vw']


# Analysis

In [197]:
ticker = 'IRS'
divs = get_dividend_dates(ticker)

## Exit Day Cost

In [198]:
for row in divs: 
    if 'profit' not in row.keys():
        ex_div_date = row['ex_dividend_date']
        if pd.to_datetime(ex_div_date) <= datetime.today():
            try:
                profit = get_day_delta(ticker, ex_div_date )
                row['profit'] = profit
            except Exception as e:
                e
                
final  = []
for r in divs: 
    if 'profit' in r.keys():
        final.append(r)

data = pd.DataFrame(final)

In [199]:
(data['cash_amount'] - data['profit']).mean()

np.float64(1.1023141999999997)

### Exit Day + 1 Cost

In [207]:
data['profit'].mean()

np.float64(-0.34955859999999966)